# HOMEWORK 3

As a first step, we import all the libraries and packages we need for this homework.
We define a function that install the packages needed, so if your laptot doesn't have it, it starts the installation.

In [5]:
def install(package):
    try:
        import subprocess
        import sys
        subprocess.call([sys.executable, "-m", "pip", "install", package])
        print("Library installed")
    except:
        print("Cannot install the library")

In [6]:
try:
    from langdetect import detect
except:
    print("The library LanfDetect is not in the repository, I will install it")
    install('langdetect')
    from langdetect import detect

try:
    from texttable import Texttable
except:
    print("The library TextTable is not in the repository, I will install it")
    install('texttable')
    from texttable import Texttable


In [7]:
import pandas as pd
import unicodecsv as csv # We also import the unicodecsv library to handle any Unicode objects in the CSV report...as csv.
import nltk
import string
import math
import numpy as np

from nltk import word_tokenize
from nltk.corpus import stopwords
from langdetect import detect

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##STEP 1 - DATA

First of all we have to charge our dataset:

We do this step dropping the column 'Unnamed: 0' , every row without description, with a description that has only a point or the '\\' characters.
Then, we decide to remove the newline character and, after this step, we initialize the language column with a 'nd' value; we will use this column to know the language of the record of our dataset.

In [8]:
DIR = 'C:/Users/david/Desktop/DATA SCIENCE/AMD/documents/' # data/docs/

df = pd.read_csv("Airbnb_Texas_Rentals.csv", sep=",") # we read the dataset

In [9]:
df = df.drop('Unnamed: 0',axis = 1) #this column is useless for our analisys
df = df[pd.notnull((df['description'])) & (df['description'] != "\\" ) & (df['description'] != "." )] #If the row has no description, for us is useless, so we drop it
df = df.replace(r'\\n',' ', regex=True) #Remove all the new lines in the dataframe
df['language'] = 'nd'
df.head()

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,language
0,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...,nd
1,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...,nd
2,$59,1,Houston,January 2017,'River house on island close to the city' A w...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...,nd
3,$60,1,Bryan,February 2016,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...,nd
4,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...,nd


STEP BONUS - DEFINE A NEW SCORE

In [1]:
import folium
from geopy import distance
import re

In [2]:
m = folium.Map(location = [32.78815, -96.7954], zoom_start = 6)
m

In [3]:
folium.Circle(
    radius = 10000,
    location = [32.78815, -96.7954],
    popup = 'The Waterfront',
    color = 'crimson',
    fill = True,
).add_to(m)

m

In [10]:
Coords = {}
for doc in range(len(df)):
    f = open('C:/Users/david/Desktop/DATA SCIENCE/AMD/documents/doc_'+str(doc)+'.tsv', 'r', encoding = 'utf-8')
    for line in f:
        ou = line.strip().split('\t')
        break
    if ou[8] not in Coords:
        if ou[6] != 'nan' and ou[7] != 'nan':
            Coords[ou[8]] = [float(ou[5]),ou[6]]

In [11]:
Coords

{'https://www.airbnb.com/rooms/18520444?location=Cleveland%2C%20TX': [30.0201379199512,
  '-95.29399600425128'],
 'https://www.airbnb.com/rooms/17481455?location=Cibolo%2C%20TX': [29.503067675606104,
  '-98.4476879378504'],
 'https://www.airbnb.com/rooms/16926307?location=Beach%20City%2C%20TX': [29.829352227214898,
  '-95.0815494887563'],
 'https://www.airbnb.com/rooms/11839729?location=College%20Station%2C%20TX': [30.6373042787676,
  '-96.33784597296308'],
 'https://www.airbnb.com/rooms/17325114?location=Colleyville%2C%20TX': [32.7470973543511,
  '-97.2864343970125'],
 'https://www.airbnb.com/rooms/14466133?location=Cleveland%2C%20TX': [30.370455202614004,
  '-95.38531903586781'],
 'https://www.airbnb.com/rooms/12491762?location=Cedar%20Creek%2C%20TX': [30.109838308143,
  '-97.47341691286819'],
 'https://www.airbnb.com/rooms/18977363?location=Cleburne%2C%20TX': [32.6896114095237,
  '-97.29816086056499'],
 'https://www.airbnb.com/rooms/17559848?location=Bayside%2C%20TX': [28.1037257224

In [12]:
def google_maps(lat, lon):
    r = 10 #in km
    for k, v in Coords.items():
        P1 = (32.78815, -96.7954)
        lat1 = v[0]
        lon1 = v[1]
        P2 = (lat1, lon1) 
        if (distance.distance(P1, P2).km) < r:
            tooltip = 'Click me!'
            folium.Marker(location = [float(P2[0]),float(P2[1])], 
                          popup = folium.Popup('<i>Mt. Hood Meadows</i>'), 
                          icon = folium.Icon(color = 'blue', icon = 'home')
                         ).add_to(m)
            
        else:
            pass

In [13]:
google_maps(32.78815, -96.7954)

In [14]:
m
